In [21]:
import os
import csv
import pandas as pd
import re
import matplotlib.pyplot as plt
import subprocess
import dendropy
from dendropy.calculate import treecompare
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import Bio
import glob
%matplotlib inline
%store -r

In [23]:
slurm = """#!/usr/bin/bash
#input file name in command line
#SBATCH --job-name=new      # Job name
#SBATCH --mail-type=END,FAIL         # Mail events (NONE, BEGIN, END, FAIL, ALL)
#SBATCH --mail-user=xxxxx@email.edu    # Where to send mail.  Set this to your email address
#SBATCH --account=xxxx
#SBATCH --qos=xxxx
#SBATCH --ntasks=1                  # Number of MPI tasks (i.e. processes)
#SBATCH --cpus-per-task=2            # Number of cores per MPI task
#SBATCH --nodes=1                    # Maximum number of nodes to be allocated
#SBATCH --ntasks-per-node=1         # Maximum number of tasks on each node
#SBATCH --mem=8gb          # Memory (i.e. RAM) per processor
#SBATCH --time=3-00:00:00              # Wall time limit (days-hrs:min:sec)
#SBATCH --output=mpi_test_%j.log     # Path to the standard
"""

In [7]:
df = pd.read_csv("./SNAP_3/sample_101123.list",sep="\t",header=0)

## Build an index for SNAP aligner

In [ ]:
script = """ml snap/2.0.3
snap-aligner index VescaGenome/Fragaria_vesca_v6_genome.fasta.gz VescaGenome/Fvesca"""
subprocess.run(script, shell = True)


## Align diploid species to F.vesca genome using SNAP aligner

In [ ]:
index_dir='VescaGenome/Fvesca'
for index, row in df.iterrows():
    if os.path.isfile('./SNAP_3/{}.bam'.format(row[0])):
        print('./SNAP_3/{}.bam exists'.format(row[0]))
    else:    
        if pd.isna(row['File3']) == True:
            script = slurm + "\nml snap/2.0.3" + "\nsnap-aligner paired {index_dir} {file1} {file2} -pre- -so -o ./SNAP_3/{id}.bam".format(index_dir = index_dir,file1 = row['File1'],file2=row['File2'],id=row[0])
        else:
            script = slurm + "\nml snap/2.0.3" + "\nsnap-aligner paired {index_dir} {file1} {file2} {file3} {file4} -pre- -so -o ./SNAP_3/{id}.bam".format(index_dir = index_dir,file1 = row['File1'],file2=row['File2'],
                                                                                                                                        file3=row['File3'],file4=row['File4'],id=row[0])
        script_file = "{}_SNAP.sh".format(row[0])
        with open(script_file, "w") as f:
            f.write(script)
        result = subprocess.run(['sbatch', script_file])

## Alternative pipeline for sorting and indexing 

In [ ]:
index_dir='VescaGenome/Fvesca'
for index, row in df.iterrows():
    if os.path.isfile('./SNAP_3/{}.bam'.format(row[0])):
        print('./SNAP_3/{}.bam exists'.format(row[0]))
    else:    
        if pd.isna(row['File3']) == True:
            script = slurm + "\nml snap/2.0.3" + "\nsnap-aligner paired {index_dir} {file1} {file2} -o ./SNAP_3/{id}.bam".format(index_dir = index_dir,file1 = row['File1'],file2=row['File2'],id=row[0])
        else:
            script = slurm + "\nml snap/2.0.3" + "\nsnap-aligner paired {index_dir} {file1} {file2} {file3} {file4} -o ./SNAP_3/{id}.bam".format(index_dir = index_dir,file1 = row['File1'],file2=row['File2'],
                                                                                                                                        file3=row['File3'],file4=row['File4'],id=row[0])
        script = script + "\nml samtools" + "\nsamtools sort ./SNAP_3/{id}.bam > ./SNAP_3/{id}_sort.bam ".format(id=row[0]) + \
            "\nmv ./SNAP_3/{id}_sort.bam ./SNAP_3/{id}.bam".format(id=row[0]) + \
            "\nsamtools index ./SNAP_3/{id}.bam".format(id=row[0])
        script_file = "{}_SNAP.sh".format(row[0])
        with open(script_file, "w") as f:
            f.write(script)
        result = subprocess.run(['sbatch', script_file])

## Simulate short reads for subgenomes

In [ ]:
# generate reads for subgenomes
ml="\nml seqkit" + "\nml art"
# grep subgenome
for subgenome, ID in zip(["C","D"],["C","D"]):
    for file in ["./Genomes/Fchil_hap1.fa.gz","./Genomes/Fvirg_hap1.fa.gz"]:
        prefix = re.match(".+(F.*)_.*",file).group(1)
        output = "{prefix}_{ID}.fasta".format(prefix = prefix,ID=ID)
        getsubfasta = "\nseqkit grep -n -r -p {subgenome} {file} > {output} ".format(subgenome=subgenome,file=file, output=output)
        ##get names for the simReads
        out_fastq = "{prefix}_{ID}_sim".format(prefix = prefix,ID=ID)
        simreads = "\nart_illumina -ss HSXt -p -l 150 -f 40 -m 200 -s 0 -i {} -o {} ".format(output,out_fastq)
        script = slurm + ml+getsubfasta+simreads
        script_file = "{}_SNAP.sh".format(out_fastq)
        with open(script_file, "w") as f:
            f.write(script)
        subprocess.run(['sbatch',script_file])

In [ ]:
# generate reads for subgenomes
ml="\nml seqkit" + "\nml art"
# grep subgenome
for subgenome, ID in zip(["A","B"],["A","B"]):
    for file in ["./Genomes/Fchil_hap1.fa.gz","./Genomes/Fvirg_hap1.fa.gz"]:
        prefix = re.match(".+(F.*)_.*",file).group(1)
        output = "{prefix}_{ID}.fasta".format(prefix = prefix,ID=ID)
        getsubfasta = "\nseqkit grep -n -r -p {subgenome} {file} > {output} ".format(subgenome=subgenome,file=file, output=output)
        ##get names for the simReads
        out_fastq = "{prefix}_{ID}_sim".format(prefix = prefix,ID=ID)
        simreads = "\nart_illumina -ss HSXt -p -l 150 -f 40 -m 200 -s 0 -i {} -o {} ".format(output,out_fastq)
        script = slurm + ml+getsubfasta+simreads
        script_file = "{}_SNAP.sh".format(out_fastq)
        with open(script_file, "w") as f:
            f.write(script)
        subprocess.run(['sbatch',script_file])

In [ ]:
# for FL158925
ml="ml seqkit" + "\nml art"
chrCb = ["Fvb" + item + "_B\n" for item in ["1-1","2-1","3-1","4-1","5-4","6-2","7-4"]] 
chrCa = ["Fvb" + item + "_B\n" for item in ["1-3","2-3","3-3","4-2","5-2","6-4","7-1"]]
chrA = ["Fvb" + item + "_B\n" for item in ['1-4','2-2','3-4','4-3','5-1','6-1','7-2']] 
chrB = ["Fvb" + item + "_B\n" for item in ['1-2','2-4','3-2','4-4','5-3','6-3','7-3']] 

# grep subgenome
for subgenome,ID in zip([chrCb,chrCa,chrA,chrB],['Cb','Ca','A','B']):
    file = '{}.chr_list'.format(ID) 
    with open(file,"w") as f:
        f.writelines(subgenome)
    output = "{prefix}_{ID}.fasta".format(prefix = 'Fxa',ID=ID)
    getsubfasta = "\nseqkit grep -n -f {file} /orange/seonghee/15.89-25_Genome/Beauty.hap.fasta > {output} ".format(file=file, output=output)
    ##get names for the simReads
    out_fastq = "{prefix}_{ID}_sim".format(prefix = 'Fxa',ID=ID)
    simreads = "\nart_illumina -ss HSXt -p -l 150 -f 40 -m 200 -s 0 -i {} -o {} ".format(output,out_fastq)
    script = slurm + ml+getsubfasta+simreads
    script_file = "{}.sh".format(out_fastq)
    with open(script_file, "w") as f:
        f.write(script)
    subprocess.run(['sbatch',script_file])

## Variant calling 

In [ ]:
ml="\nml gatk"
haplocaller_script = """
gatk --java-options "-Xmx32g" HaplotypeCaller  \
         -R /orange/whitaker/zhen/Polyorigin/VescaGenome/Fragaria_vesca_v6_genome.fasta \
         --native-pair-hmm-threads 8 \
         -L chr.list \
         -I {file} \
         -O {id}.g.vcf.gz \
         -ERC GVCF
"""

all_files_and_dirs = os.listdir("./SNAP_3/")
bam_files = [f for f in all_files_and_dirs if re.fullmatch(".+bam",f) ]
for bam_file in bam_files:
    id1 = re.match("(^.+).bam",bam_file).group(1)
    script = haplocaller_script.format(file = "./SNAP_3/" + bam_file, id = "./GVCF/"+ id1)
    script = slurm + ml + script
    script_file = "{}_hapcall.sh".format(id1)
    with open(script_file, "w") as f:
        f.write(script)
    subprocess.run(['sbatch',script_file])

## Merge GVCF

In [ ]:
ml="\nml gatk"
CombineGVCFs_script = """
gatk CombineGVCFs \
   -R /blue/whitaker/fanzhen/asm/farr1.fa \
   --variant GVCF/ERR2003066.g.vcf.gz \
   --variant GVCF/ERR2008847.g.vcf.gz \
   --variant GVCF/ERR2008776.g.vcf.gz \
   --variant GVCF/ERR2008777.g.vcf.gz \
   --variant GVCF/ERR2008844.g.vcf.gz \
   --variant GVCF/ERR2008845.g.vcf.gz \
   -O GVCF/PMi.g.vcf.gz"""
script = slurm + ml + CombineGVCFs_script
script_file = "CombineGVCFs.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

## GenomicsDBImport

In [ ]:
df["GVCF"] = "GVCF/" + df.iloc[:,0] + ".g.vcf.gz"
df_map = df.loc[:,["New Name","GVCF"]]
df_map.to_csv('GVCFmap.csv', index=False, header=False, quoting=3,sep = "\t")

In [ ]:
ml="\nml gatk"
genomicDB_script = """\ngatk --java-options "-Xmx100g -Xms100g" \
       GenomicsDBImport \
       --genomicsdb-workspace-path {db_path} \
       -L chr.list \
       --sample-name-map {map} \
       --reader-threads 7 \
        --max-num-intervals-to-import-in-parallel 10 \
        --genomicsdb-shared-posixfs-optimizations \
        --batch-size 16 \
        --tmp-dir /orange/whitaker/zhen/popgen/tmp/
"""
script = slurm + ml + genomicDB_script.format(map = 'GVCFmap.csv', db_path= "GDB/")
script_file = "GenomicsDBImport.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

## GenotypeGVCFs

In [ ]:
ml="\nml gatk"
GVCF_script = """
gatk --java-options "-Xmx32g" GenotypeGVCFs \
   -R /orange/whitaker/zhen/Polyorigin/VescaGenome/Fragaria_vesca_v6_genome.fasta\
   -V gendb://GDB \
   -O raw.40w.vcf.gz \
   -L chr.list \
   --genomicsdb-shared-posixfs-optimizations
   """
script = slurm + ml + GVCF_script
script_file = "GVCF.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

## Hard filtering of SNPs

In [ ]:
##Filter using GATK
ml="\nml gatk"
Filtering_script = """
gatk VariantFiltration \
    -V raw.40w.vcf.gz \
   --filter-name "QUAL30" -filter "QUAL < 30.0" \
   --filter-name "QD2"  -filter "QD < 2.0"  \
   --filter-name "FS55" -filter "FS > 55.0"  \
        --filter-name "SOR3" -filter "SOR > 3.0"  \
         --filter-name "MQ55" -filter "MQ < 55.0"  \
        --filter-name "MQR_2" -filter "MQRankSum < -2.0"  \
        -O filter.40w.vcf.gz
"""
script = slurm + ml + Filtering_script
script_file = "Filtering_script.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])


In [ ]:
##Additional filtering steps
ml = "\nml bcftools"
filter1 = """\nbcftools view -i'FILTER="PASS" && F_MISSING<0.3' --no-update -Oz < filter.40w.vcf.gz  > passed.40w.vcf.gz"""
filterSNP = """\nbcftools view -i'TYPE="snp" && N_ALT=1' -Oz  passed.40w.vcf.gz  > SNP.40w.vcf.gz"""
indexSNP = """\nbcftools index SNP.40w.vcf.gz"""
#filterLD = """\nbcftools view -R ../popgen/farr1.intron.only.bed -Oz SNP.38w.vcf.gz > SNP.38w.intron.vcf.gz 
#bcftools +prune -m 0.4 -w 1000 --nsites-per-win-mode maxAF -Oz SNP.38w.intron.vcf.gz -o SNP.intron.04.38w.vcf.gz"""
script = slurm + ml + filter1 + filterSNP + indexSNP 
#script = slurm + ml +  filterSNP + indexSNP + filterLD
script_file = "Filtering2_script.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

## Concatenated species tree

In [ ]:
vcf2phy = """
ml python
python ../popgen/vcf2phylip/vcf2phylip/vcf2phylip.py -i SNP.40w.vcf.gz --nexus"""
iqtree = """
ml iq-tree
iqtree2 -s SNP.40w.min4.phy -B 1000 -T AUTO --prefix SNP.40w -alrt 1000 -m MFP -T AUTO -redo
"""
script = slurm + vcf2phy + iqtree
script_file = "iqtree_full_script.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

## 10k-variant window, Astral and SVDq trees

In [ ]:
##produce a partition file 
l1=list(range(1,1611))
l2=list(range(1,16104631,10000))
l3=list(range(10000,16114631,10000))
l4 = ["DNA, part" + str(a) + " = " + str(b) + "-" + str(c) for a,b,c in zip(l1,l2,l3)]
with open('partition.txt', 'w') as file:
    for item in l4:
        file.write(f"{item}\n")

In [ ]:
##build gene trees
ml = "\nml iq-tree"
tree = """\niqtree2 -s SNP.40w.min4.phy -S partition.txt --prefix Genetree/loci -T AUTO"""
script = slurm + ml + tree
script_file = "genetree_script.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

In [ ]:
##ASTRAL tree
"java -jar astral.5.7.8.jar -i in.tree -o out.tre 2>out.log"

In [ ]:
##SVD quartz
##build nex file 
script_file = "species.nex"
start = """#NEXUS
begin paup;
        cd *;
        set hashcomment=y;
end;
execute {alnfile};
begin sets;
charpartition lociset =
"""
with open(script_file, "w") as f:
    f.write(start.format(alnfile="SNP.40w.min4.nexus"))
l1=list(range(1,1611))
l2=list(range(1,16104631,10000))
l3=list(range(10000,16114631,10000))
l4 = [str(a) + " : " + str(b) + "-" + str(c) + "," for a,b,c in zip(l1,l2,l3)]
with open(script_file, 'a') as file:
    for item in l4:
        file.write(f"{item}\n")
end = """end;
begin paup;
outgroup Rchinensis;
svdq loci=lociset bootstrap=multilocus showScores=no seed=123 treeFile=SVDQ/species_SVDQ.tre ;
#save bootstrap values
rootTrees rootMethod=outgroup;
contree all/strict=no majrule=yes usetreewts=yes treefile=svdq.boot.tre;
end;
quit;
"""
with open(script_file, "a") as f:
    f.write(end)
##change "," after last charpartition lociset to ";"

In [ ]:
##run svdq
script = """ml paup/4.0a168
paup species.nex"""
script = slurm + script
script_file = "sdvq.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

In [8]:
##run astral
script = """ml astral
astral -i Genetree/loci.treefile -o Astral/astral.tre 2>Astral/astral.log"""
script = slurm + script
script_file = "astral.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

Submitted batch job 34656631


CompletedProcess(args=['sbatch', 'astral.sh'], returncode=0)

## add gCF to each branch of species tree

In [ ]:
##Gene concordance factor (gCF)
ml = "\nml iq-tree/2.2.2.7"
tree = """iqtree2 -t contree/SNP.40w.treefile --gcf Genetree/loci.treefile --prefix contree/concord"""
script = slurm + ml + tree
script_file = "gcf_script.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])


## Treemix

In [ ]:
script = """ml bcftools
bcftools view -S treemix/short.list -Oz SNP.40w.vcf.gz > SNP.32w.vcf.gz"""
subprocess.run(script,shell = True)

In [ ]:
#bash command s
##./vcf2treemix.sh ../SNP.32w.vcf.gz name.clust
##run treemix with -k 500
ml = "\nml treemix"
script = "\ntreemix -i treemix/SNP.32w.treemix.frq.gz -k 500 -m {} -seed {} -bootstrap -root Rchinensis -o treemix/treemix_073024/{}_s{}.treemix"
for i in range(2,4):
    for s in range(1,21):
        script1 = slurm + ml + script.format(i,s,i,s)
        script_file = "treemix_script_{}_s{}.sh".format(i,s)
        with open(script_file, "w") as f:
            f.write(script1)
        subprocess.run(['sbatch',script_file])
##has to run treemix with the right tf after getting the optimum migration edges

## PhyloNet tree

In [16]:
from Bio import Phylo
import re
# some samples will be removed from tree.
prune_list = ['bracteata_1','bracteata_E1','bracteata_E2','viridis_3','vesca_3','bracteata_3','mandschurica_1','mandschurica_2','mandschurica_3']
# Function to convert tree file to NEXUS format and remove TaxLabels
def convert_to_nexus(input_file, output_file):
    # Read the trees from the input file
    trees = list(Phylo.parse(input_file, 'newick'))
    ##prune tree using a list of taxa
    subtrees = list()
    for tree in trees:
        for taxon in prune_list:
            if tree.find_any(name=taxon):
                tree.prune(taxon)
    # Write the trees to a temporary NEXUS file
    temp_output_file = 'temp_output_file.nex'
    Phylo.write(trees, temp_output_file, 'nexus')
    
    # Read the temporary NEXUS file
    with open(temp_output_file, 'r') as file:
        nexus_content = file.read()
    
    # Remove the TaxLabels block using regex
    nexus_content = re.sub(r'Begin Taxa;.*?End;', '', nexus_content, flags=re.DOTALL)
    
    # Write the modified content to the final output file
    with open(output_file, 'w') as file:
        file.write(nexus_content)

# Example usage
input_file = './Genetree/loci.treefile'  # replace with your input file name
output_file = './PhyloNet/locitree_noMan.nex'  # replace with your desired output file name
convert_to_nexus(input_file, output_file)
phy_script = """
BEGIN PHYLONET;
InferNetwork_MP (all) 3 -n 3 -pl 16 -di -a <iinumae:iinumae_1,iinumae_2,iinumae_3;nilgerrensis:nilgerrensis_1,nilgerrensis_2;nipponica:nipponica_1,nipponica_2;bracteata1:americana,bracteata_F;bracteata2:bracteata_2,bracteata_4,bracteata_5;vesca:vesca_1,vesca_2,alba;viridis:viridis_1,viridis_2;octoploid_A:ananassa_A,chiloensis_A,virginiana_A;octoploid_B:ananassa_B,chiloensis_B,virginiana_B;octoploid_Ca:ananassa_Ca,chiloensis_Ca,virginiana_Ca;octoploid_Cb:ananassa_Cb,chiloensis_Cb,virginiana_Cb;Pmicrophylla:Pmicrophylla;Rchinensis:Rchinensis> ;
InferNetwork_MPL (all) 3 -pl 16 -di -a <iinumae:iinumae_1,iinumae_2,iinumae_3;nilgerrensis:nilgerrensis_1,nilgerrensis_2;nipponica:nipponica_1,nipponica_2;bracteata1:americana,bracteata_F;bracteata2:bracteata_2,bracteata_4,bracteata_5;vesca:vesca_1,vesca_2,alba;viridis:viridis_1,viridis_2;octoploid_A:ananassa_A,chiloensis_A,virginiana_A;octoploid_B:ananassa_B,chiloensis_B,virginiana_B;octoploid_Ca:ananassa_Ca,chiloensis_Ca,virginiana_Ca;octoploid_Cb:ananassa_Cb,chiloensis_Cb,virginiana_Cb;Pmicrophylla:Pmicrophylla;Rchinensis:Rchinensis> ;
InferNetwork_ML_Bootstrap (all) 3 -pl 16 -di -a <iinumae:iinumae_1,iinumae_2,iinumae_3;nilgerrensis:nilgerrensis_1,nilgerrensis_2;nipponica:nipponica_1,nipponica_2;bracteata1:americana,bracteata_F;bracteata2:bracteata_2,bracteata_4,bracteata_5;vesca:vesca_1,vesca_2,alba;viridis:viridis_1,viridis_2;octoploid_A:ananassa_A,chiloensis_A,virginiana_A;octoploid_B:ananassa_B,chiloensis_B,virginiana_B;octoploid_Ca:ananassa_Ca,chiloensis_Ca,virginiana_Ca;octoploid_Cb:ananassa_Cb,chiloensis_Cb,virginiana_Cb;Pmicrophylla:Pmicrophylla;Rchinensis:Rchinensis> ;
END;
"""
with open(output_file, 'a') as file1:
    # Writing data to a file
    file1.write(phy_script)
##run PHYLONET
##multiple alleles per sample did not work 
script = """
ml phylonet
PhyloNet ./PhyloNet/locitree_noMan.nex"""
script = slurm + script
script_file = "PhyloNetM.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])


Submitted batch job 35360988


CompletedProcess(args=['sbatch', 'PhyloNetM.sh'], returncode=0)

## Phylonet with variable hybridization edges

In [21]:
from Bio import Phylo
import re
prune_list = ['bracteata_1','bracteata_E1','bracteata_E2','viridis_3','vesca_3','bracteata_3','mandschurica_1','mandschurica_2','mandschurica_3']
# Function to convert tree file to NEXUS format and remove TaxLabels
def convert_to_nexus(input_file, output_file):
    # Read the trees from the input file
    trees = list(Phylo.parse(input_file, 'newick'))
    ##prune tree using a list of taxa
    subtrees = list()
    for tree in trees:
        for taxon in prune_list:
            if tree.find_any(name=taxon):
                tree.prune(taxon)
    # Write the trees to a temporary NEXUS file
    temp_output_file = 'temp_output_file.nex'
    Phylo.write(trees, temp_output_file, 'nexus')
    
    # Read the temporary NEXUS file
    with open(temp_output_file, 'r') as file:
        nexus_content = file.read()
    
    # Remove the TaxLabels block using regex
    nexus_content = re.sub(r'Begin Taxa;.*?End;', '', nexus_content, flags=re.DOTALL)
    
    # Write the modified content to the final output file
    with open(output_file, 'w') as file:
        file.write(nexus_content)

# Example usage
input_file = './Genetree/loci.treefile'  # replace with your input file name
output_file = './PhyloNet/locitree_MPL.nex'  # replace with your desired output file name
convert_to_nexus(input_file, output_file)
phy_script = """
BEGIN PHYLONET;
InferNetwork_MPL (all) 2 -pl 16 -di -x 100 -a <iinumae:iinumae_1,iinumae_2,iinumae_3;nilgerrensis:nilgerrensis_1,nilgerrensis_2;nipponica:nipponica_1,nipponica_2;bracteata1:americana,bracteata_F;bracteata2:bracteata_2,bracteata_4,bracteata_5;vesca:vesca_1,vesca_2,alba;viridis:viridis_1,viridis_2;octoploid_A:ananassa_A,chiloensis_A,virginiana_A;octoploid_B:ananassa_B,chiloensis_B,virginiana_B;octoploid_Ca:ananassa_Ca,chiloensis_Ca,virginiana_Ca;octoploid_Cb:ananassa_Cb,chiloensis_Cb,virginiana_Cb;Pmicrophylla:Pmicrophylla;Rchinensis:Rchinensis> ;
InferNetwork_MPL (all) 3 -pl 16 -di -x 100 -a <iinumae:iinumae_1,iinumae_2,iinumae_3;nilgerrensis:nilgerrensis_1,nilgerrensis_2;nipponica:nipponica_1,nipponica_2;bracteata1:americana,bracteata_F;bracteata2:bracteata_2,bracteata_4,bracteata_5;vesca:vesca_1,vesca_2,alba;viridis:viridis_1,viridis_2;octoploid_A:ananassa_A,chiloensis_A,virginiana_A;octoploid_B:ananassa_B,chiloensis_B,virginiana_B;octoploid_Ca:ananassa_Ca,chiloensis_Ca,virginiana_Ca;octoploid_Cb:ananassa_Cb,chiloensis_Cb,virginiana_Cb;Pmicrophylla:Pmicrophylla;Rchinensis:Rchinensis> ;
END;
"""
with open(output_file, 'a') as file1:
    # Writing data to a file
    file1.write(phy_script)
##run PHYLONET
##multiple alleles per sample did not work 
script = """
ml phylonet
PhyloNet ./PhyloNet/locitree_MPL.nex"""
script = slurm + script
script_file = "PhyloNetM.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

Submitted batch job 35414335


CompletedProcess(args=['sbatch', 'PhyloNetM.sh'], returncode=0)

## Dsuite

In [ ]:
nl = pd.read_csv("treemix/name.clust",sep = "\t",header = None)
nl = nl.iloc[:,1:3]
nl.loc[nl[1]=="Rchinensis",2] = "Outgroup"
nl.to_csv("Dsuite/sets.txt",sep = "\t",index=False,header = False)

In [ ]:
ml = "\nml gcc/9.3.0 \nml dsuite"
script = "\nDsuite Dtrios -c  -o Dsuite/32w -t Dsuite/species.tre SNP.32w.vcf.gz Dsuite/sets.txt"
script1 = slurm + ml + script
script_file = "Dsuite_script.sh"
with open(script_file, "w") as f:
    f.write(script1)
subprocess.run(['sbatch',script_file])

In [ ]:
# Define the file path
file_path = 'Dsuite/32w_tree.txt'
file_path1 = 'Dsuite/32w_curated_tree.txt'
# Read the file
with open(file_path, 'r') as file:
    content = file.read()
# Remove the special character
content = content.replace('_', '')
# Write the modified content back to the file
with open(file_path1, 'w') as file:
    file.write(content)
# Define the file path
file_path = 'Dsuite/species.tre'
file_path1 = 'Dsuite/species_curated.tre'
# Read the file
with open(file_path, 'r') as file:
    content = file.read()
# Remove the special character
content = content.replace('_', '')
# Write the modified content back to the file
with open(file_path1, 'w') as file:
    file.write(content)

In [ ]:
## Fbranch 
ml = "\nml gcc/9.3.0 \nml dsuite"
script = "\nDsuite Fbranch Dsuite/species.tre Dsuite/32w_tree.txt > Dsuite/32w_Fbranch.txt"""
subprocess.run( ml+script, shell= True)


In [ ]:
%run ../popgen/Dsuite/Dsuite/utils/dtools.py  Dsuite/32w_Fbranch.txt Dsuite/species.tre

## Dating using orthologs from whole genome assemblies 

In [ ]:
##need to extract four subgenomes from whole genome assemblies
##remove gene trees place viridis and octoploid genome together
for s in ["A", "B","C","D"]:
    with open(f'OtherGenomes/sub_{s}.list', 'w') as file:
        # Loop through numbers 1 to 7
        for i in range(1, 8):
            # Write each number followed by 'A' to the file, with each on a new line
            file.write(f"-{i}{s}-\n")
    script = f"""ml seqkit
    seqkit grep -n -r -f OtherGenomes/sub_{s}.list OtherGenomes/Fchil_hap1.pep.fa -o OtherGenomes/Fchil_{s}.fa"""
    subprocess.run(script,shell = True)

In [ ]:
orth = """
ml orthofinder
orthofinder -f OtherGenomes
"""
script = slurm + orth
script_file = "Orthofinder_script.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

In [ ]:
# Define the directory containing the tree files
directory = r"/orange/vwhitaker/zhen/Polyorigin/OtherGenomes/OrthoFinder/Results_Nov14_1/Gene_Trees/"
# Create a dictionary to store trees
trees = {}
##only retain single copy genes
single_cp = os.listdir(r"/orange/vwhitaker/zhen/Polyorigin/OtherGenomes/OrthoFinder/Results_Nov14_1/Single_Copy_Orthologue_Sequences/")
single_cp1 = [re.sub(".fa","_tree.txt",string) for string in single_cp]
# Iterate over each file in the directory
for filename in single_cp1:
    if filename.endswith(".txt"):  # Assuming the files are in Newick format and have .tre extension
        # Construct the full file path
        filepath = os.path.join(directory, filename)
        
        # Load the tree from the file
        tree = dendropy.Tree.get(
            path=filepath,
            schema="newick",  # Change this if your tree files are in a different format
            preserve_underscores = True
        )

        # Store the tree with the filename as the key
        trees[filename] = tree

In [ ]:
def find_first_occurrence_with_in(lst, pattern):
    for item in lst:
        if pattern in item:
            return item
    return None  # Pattern not found in any element

In [ ]:
non_adm = []
for id1, tree in trees.items():
    print(id1)
    labels = [node.label for node in tree.taxon_namespace]
    pat_4 = ["Potentilla_proteins","Fviridis2_pep", "Fragaria_vesca_v4", "Fvirg_A"]
    t_labels = [find_first_occurrence_with_in(labels, pat) for pat in pat_4]
    tree1 = tree.extract_tree()
    tree1.retain_taxa_with_labels(t_labels,update_bipartitions=True)
    tns=tree1.taxon_namespace
    t1 = "('{}',('{}',('{}','{}')));".format(t_labels[0],t_labels[1],t_labels[2],t_labels[3])
    Atree1 = dendropy.Tree.get(
            data=t1,
            schema='newick',
            taxon_namespace=tns)
    if treecompare.symmetric_difference(Atree1, tree1)==0:
        non_adm.append(id1)

In [ ]:
#save outputs to file
non_adm_df = pd.DataFrame(non_adm)
single_orthlogs = non_adm_df[0].str.extract('(OG.+)_.+')
single_orthlogs.to_csv('singleOrthologsWOadmixture.csv', index=False)

In [ ]:
##change names, laad all functions
New_names=["Fchil_A","Fchil_B","Fchil_C","Fchil_D","Fmanchurica","Fnilgerrensis", "Fnipponica","Fiinumae", "Fvesca_H",
         "Fvesca_R","Fvirg_A","Fvirg_B","Fvirg_C","Fvirg_D","Fviridis1","Fviridis2","Pmicrantha","Rchinensis"]
single_orth = pd.read_csv('singleOrthologsWOadmixture.csv')
single_arr = single_orth.to_numpy()
def modify_sequence_names(records, new_names):
    for i, record in enumerate(records):
        if i < len(new_names):
            record.id = new_names[i]
            record.description = ""
        else:
            break           

In [ ]:
##create raw fasta files for non-admixed orthlogs 
dir = r"/orange/vwhitaker/zhen/Polyorigin/OtherGenomes/OrthoFinder/Results_Nov14_1/Single_Copy_Orthologue_Sequences/"
out_dir = r"/orange/vwhitaker/zhen/Polyorigin/speciestree/raw/"
for orth in single_arr:
    fasta_file = os.path.join(dir, orth[0] + ".fa")
    records = list(SeqIO.parse(fasta_file, "fasta"))
    modify_sequence_names(records, New_names)
    with open(out_dir+orth[0]+".fasta", "w") as output_handle:
        SeqIO.write(records, output_handle, "fasta")
    print("Modified FASTA file created:", orth+".fasta")

In [ ]:
##MAFFT alignment 
ml = """
ml mafft
"""
single_fa = os.listdir(out_dir)
out_mafft =  r"/orange/vwhitaker/zhen/Polyorigin/speciestree/MAFFT/"
for file in single_fa:
    script = slurm + ml+"mafft {} > {}".format(out_dir+file,out_mafft+file) 
    script_file = file + ".sh"
    with open(script_file, "w") as f:
        f.write(script)
    subprocess.run(['sbatch',script_file])

In [ ]:
##create MAFFT alignment for all single copy orthologs
single_cp = os.listdir(r"/orange/vwhitaker/zhen/Polyorigin/OtherGenomes/OrthoFinder/Results_Nov14_1/Single_Copy_Orthologue_Sequences/")
dir = r"/orange/vwhitaker/zhen/Polyorigin/OtherGenomes/OrthoFinder/Results_Nov14_1/Single_Copy_Orthologue_Sequences/"
out_dir = r"/orange/vwhitaker/zhen/Polyorigin/speciestree/singlecopy/"
for file in single_cp:
    records = list(SeqIO.parse(dir+file, "fasta"))
    modify_sequence_names(records, New_names)
    with open(out_dir+file, "w") as output_handle:
        SeqIO.write(records, output_handle, "fasta")
    print("Modified FASTA file created:", file)

In [ ]:
##MAFFT alignment 
ml = """
ml mafft
"""
single_fa = os.listdir(out_dir)
out_mafft =  r"/orange/vwhitaker/zhen/Polyorigin/speciestree/MAFFT_singlecopy/"
for file in single_fa:
    script = slurm + ml+"mafft {} > {}".format(out_dir+file,out_mafft+file) 
    script_file = file + ".sh"
    with open(script_file, "w") as f:
        f.write(script)
    subprocess.run(['sbatch',script_file])

In [ ]:
##concatenate non-admixed MAFFT outputs
# Dictionary to store sequences by name
concatenated_sequences = {}
input_files = os.listdir(out_mafft)
output_file = r"/orange/vwhitaker/zhen/Polyorigin/speciestree/concatenated_singlecopy.fasta"
# Iterate through input files and store sequences by name
for input_file in input_files:
    with open(out_mafft+input_file, "r") as fasta_file:
        for record in SeqIO.parse(fasta_file, "fasta"):
            seq_name = record.id
            if seq_name not in concatenated_sequences:
                concatenated_sequences[seq_name] = ""
            concatenated_sequences[seq_name] += str(record.seq)

# Write the concatenated sequences to the output file
with open(output_file, "w") as output_handle:
    for seq_name, sequence in concatenated_sequences.items():
        output_handle.write(f">{seq_name}\n{sequence}\n")

print("Concatenated FASTA file created:", output_file)

In [4]:
##concatenate nonadmixed multiple MAFFT outputs
# Dictionary to store sequences by name
concatenated_sequences = {}
out_mafft =  r"/orange/vwhitaker/zhen/Polyorigin/speciestree/MAFFT/"
input_files = os.listdir(out_mafft)
output_file = r"/orange/vwhitaker/zhen/Polyorigin/speciestree/concatenated_noAdmix.fasta"
# Iterate through input files and store sequences by name
for input_file in input_files:
    with open(out_mafft+input_file, "r") as fasta_file:
        for record in SeqIO.parse(fasta_file, "fasta"):
            seq_name = record.id
            if seq_name not in concatenated_sequences:
                concatenated_sequences[seq_name] = ""
            concatenated_sequences[seq_name] += str(record.seq)

# Write the concatenated sequences to the output file
with open(output_file, "w") as output_handle:
    for seq_name, sequence in concatenated_sequences.items():
        output_handle.write(f">{seq_name}\n{sequence}\n")

print("Concatenated FASTA file created:", output_file)

Concatenated FASTA file created: /orange/vwhitaker/zhen/Polyorigin/speciestree/concatenated_noAdmix.fasta


In [ ]:
##build a new fasttree using the non-admixed genes
single_orth = pd.read_csv('singleOrthologsWOadmixture.csv')
single_arr = single_orth.to_numpy()
dir1=r"/orange/vwhitaker/zhen/Polyorigin/OtherGenomes/OrthoFinder/Results_Nov14_1/Gene_Trees/"

# Output file name
output_file = r"/orange/vwhitaker/zhen/Polyorigin/speciestree/Astral/nonAdmix_gene_trees.txt"

New_names=["Fchil_A","Fchil_B","Fchil_C","Fchil_D","Fmanchurica","Fnilgerrensis", "Fnipponica","Fiinumae", "Fvesca_H",
         "Fvesca_R","Fvirg_A","Fvirg_B","Fvirg_C","Fvirg_D","Fviridis1","Fviridis2","Pmicrantha","Rchinensis"]
patterns_and_replacements = [
    ('Fchil_A[^:]+:', 'Fchil_A:'),
    ('Fchil_B[^:]+:', 'Fchil_B:'),
    ('Fchil_C[^:]+:', 'Fchil_C:'),
    ('Fchil_D[^:]+:', 'Fchil_D:'),
    ('Fmanchurica[^:]+:', 'Fmanchurica:'),
    ('Fnilgerrensis[^:]+:', 'Fnilgerrensis:'),
    ('Fnip[^:]+:', 'Fnipponica:'),
    ('Fragaria_iinumae[^:]+:', 'Fiinumae:'),
    ('Fragaria_vesca_v4[^:]+:', 'Fvesca_H:'),
    ('Fvesca_CFRA2339[^:]+:', 'Fvesca_R:'),
    ('Fvirg_A[^:]+:', 'Fvirg_A:'),
    ('Fvirg_B[^:]+:', 'Fvirg_B:'),
    ('Fvirg_C[^:]+:', 'Fvirg_C:'),
    ('Fvirg_D[^:]+:', 'Fvirg_D:'),
    ('Fviridis.pep[^:]+:', 'Fviridis1:'),
    ('Fviridis2.pep[^:]+:', 'Fviridis2:'),
    ('Potentilla_proteins[^:]+:', 'Pmicrantha:'),
    ('Rosa_chinensis[^:]+:', 'Rchinensis:'),
]
# Function to perform the substitutions
def substitute_patterns(input_string, patterns_and_replacements):
    for pattern, replacement in patterns_and_replacements:
        input_string = re.sub(pattern, replacement, input_string)
    return input_string
# Open the output file in write mode
with open(output_file, "w") as output:
    for file_name in single_arr:
        try:
            # Open each input file in read mode
            filename= dir1 + file_name[0]+"_tree.txt"
            with open(filename, "r") as input_file:
                # Read the contents of the input file
                file_contents = input_file.read()
                #substitution
                file_contents = substitute_patterns(file_contents, patterns_and_replacements)
                # Write the contents to the output file
                output.write(file_contents+"\n")
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except Exception as e:
            print(f"Error reading {file_name}: {str(e)}")

## R8S with bootstrapping results

In [ ]:
##create bootstrap datasets
import random
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord

# Load the alignment from a PHYLIP file
alignment = AlignIO.read("/orange/vwhitaker/zhen/Polyorigin/speciestree/concatenated.fasta", "fasta")
num_columns = alignment.get_alignment_length()
num_replicates = 100

# Generate 100 bootstrap replicates
for i in range(1, num_replicates + 1):
    # Create a new alignment by resampling columns with replacement
    bootstrap_columns = [random.randint(0, num_columns - 1) for _ in range(num_columns)]
    bootstrap_alignment = MultipleSeqAlignment(
        [
            SeqRecord("".join(record.seq[j] for j in bootstrap_columns), id=record.id)
            for record in alignment
        ]
    )

    # Save the bootstrap replicate to a new PHYLIP file
    AlignIO.write(bootstrap_alignment, f"speciestree/r8s/bootstrap_noAdmix_{i}.phy", "phylip-relaxed")

    print(f"Bootstrap replicate {i} saved as bootstrap_noAdmix_{i}.phy")

In [ ]:
##calculate bootstrap ML tree using iqtreez
num_replicates = 100
for i in range(1, num_replicates + 1):
    ml = "\nml iq-tree"
    script = f"\niqtree -s speciestree/r8s/bootstrap_noAdmix_{i}.phy -te speciestree/nonAdmix_Astral_1.tre -m JTT+F+I+R8 -pre speciestree/nonAdmix_Astral_1.tre -o Rchinensis --prefix speciestree/r8s/boot.{i}.nonAdmix 2>speciestree/r8s/out.{i}.nonadmix.boot.log"
    script = slurm +ml + script
    script_file = f"R8S_boot_script_nonadmix.{i}.sh"
    with open(script_file, "w") as f:
        f.write(script)
    subprocess.run(['sbatch',script_file])



In [ ]:
import random
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord

# Load the alignment from a PHYLIP file
alignment = AlignIO.read("/orange/vwhitaker/zhen/Polyorigin/speciestree/concatenated_singlecopy.fasta", "fasta")
num_columns = alignment.get_alignment_length()
num_replicates = 100

# Generate 100 bootstrap replicates
for i in range(1, num_replicates + 1):
    # Create a new alignment by resampling columns with replacement
    bootstrap_columns = [random.randint(0, num_columns - 1) for _ in range(num_columns)]
    bootstrap_alignment = MultipleSeqAlignment(
        [
            SeqRecord("".join(record.seq[j] for j in bootstrap_columns), id=record.id)
            for record in alignment
        ]
    )

    # Save the bootstrap replicate to a new PHYLIP file
    AlignIO.write(bootstrap_alignment, f"speciestree/r8s/bootstrap_singlecopy_{i}.phy", "phylip-relaxed")

    print(f"Bootstrap replicate {i} saved as bootstrap_singlecopy_{i}.phy")

In [ ]:
## 100 bootstrap alignments were created using seqboot in phylip
##calculate bootstrap ML tree using iqtreez
num_replicates = 100
for i in range(1, num_replicates + 1):
    ml = "\nml iq-tree"
    script = f"\niqtree -s speciestree/r8s/bootstrap_singlecopy_{i}.phy -te speciestree/Astral/astral_tree_rooted_allsingle.txt -m JTT+F+I+R8 -pre speciestree/Astral/astral_tree_rooted_allsingle.txt -o Rchinensis --prefix speciestree/r8s/boot.{i}.singlecopy 2>speciestree/r8s/out.{i}.singlecopy.boot.log"
    script = slurm +ml + script
    script_file = f"R8S_boot_script_singlecopy.{i}.sh"
    with open(script_file, "w") as f:
        f.write(script)
    subprocess.run(['sbatch',script_file])



In [57]:
## prepare input file for R8s
from Bio import Phylo
from io import StringIO

# Load the trees
trees = []
for i in range(1, 101):  # Assuming 100 files named tree1.newick, tree2.newick, ..., tree100.newick
    tree = Phylo.read(f'speciestree/r8s/boot.{i}.singlecopy.treefile', 'newick')
    # Specify the outgroup (e.g., a specific taxon name)
    outgroup = tree.find_any(name='Rchinensis')

    # Root the tree using the outgroup
    tree.root_with_outgroup(outgroup)
    trees.append(tree)

# Write the trees to a Nexus file without a TAXA block
with open('speciestree/r8s/r8s.boot.singlecopy.nex', 'w') as nexus_file:
    nexus_file.write("#NEXUS\n\nBEGIN TREES;\n")
    for idx, tree in enumerate(trees, start=1):
        tree_str = StringIO()
        Phylo.write(tree, tree_str, "newick")
        tree_newick = tree_str.getvalue().strip()
        nexus_file.write(f"\tTREE tree_{idx} = {tree_newick}\n")
    nexus_file.write("END;\n")

r8s_com = """
begin r8s;
blformat lengths=persite nsites=647966 ultrametric=no;
MRCA node1 Pmicrantha Rchinensis;
MRCA node2 Pmicrantha Fchil_B;
MRCA node3 Fnilgerrensis Fviridis1;
MRCA node4 Fchil_A Fvesca_R;
MRCA node5 Fchil_C Fchil_D;
MRCA node6 Fiinumae Fvirg_B;
constrain taxon=node1 max_age=55;
fixage taxon=node2 age=23;
constrain taxon=node3 min_age=2.6;
divtime method=pl algorithm=tn;
showage;
profile taxon=node4 parameter=age;
profile taxon=node5 parameter=age;
profile taxon=node6 parameter=age;
describe plot=tree_description;
end;
"""
with open("speciestree/r8s/r8s.boot.singlecopy.nex", "a") as f:
    f.write(r8s_com)

##to run r8s
##command line
##r8s -b -f r8s.boot.singlecopy.nex > r8s.boot.result.singlecopy.txt

In [58]:
## prepare input file for R8s
from Bio import Phylo
from io import StringIO

# Load the trees
trees = []
for i in range(1, 101):  # Assuming 100 files named tree1.newick, tree2.newick, ..., tree100.newick
    tree = Phylo.read(f'speciestree/r8s/boot.{i}.nonAdmix.treefile', 'newick')
    # Specify the outgroup (e.g., a specific taxon name)
    outgroup = tree.find_any(name='Rchinensis')

    # Root the tree using the outgroup
    tree.root_with_outgroup(outgroup)
    trees.append(tree)

# Write the trees to a Nexus file without a TAXA block
with open('speciestree/r8s/r8s.boot.noAdmix.nex', 'w') as nexus_file:
    nexus_file.write("#NEXUS\n\nBEGIN TREES;\n")
    for idx, tree in enumerate(trees, start=1):
        tree_str = StringIO()
        Phylo.write(tree, tree_str, "newick")
        tree_newick = tree_str.getvalue().strip()
        nexus_file.write(f"\tTREE tree_{idx} = {tree_newick}\n")
    nexus_file.write("END;\n")

r8s_com = """
begin r8s;
blformat lengths=persite nsites=487377 ultrametric=no;
MRCA node1 Pmicrantha Rchinensis;
MRCA node2 Pmicrantha Fchil_B;
MRCA node3 Fnilgerrensis Fviridis1;
MRCA node4 Fchil_A Fvesca_R;
MRCA node5 Fchil_C Fchil_D;
MRCA node6 Fiinumae Fvirg_B;
constrain taxon=node1 max_age=55;
fixage taxon=node2 age=23;
constrain taxon=node3 min_age=2.6;
divtime method=pl algorithm=tn;
showage;
profile taxon=node4 parameter=age;
profile taxon=node5 parameter=age;
profile taxon=node6 parameter=age;
describe plot=tree_description;
end;
"""
with open("speciestree/r8s/r8s.boot.noAdmix.nex", "a") as f:
    f.write(r8s_com)

##to run r8s
##command line
##r8s -b -f r8s.boot.noAdmix.nex > r8s.boot.result.noAdmix.txt

# Build an Astral with non-admixed orthologs 

In [ ]:
script = """ml astral
astral -i /orange/vwhitaker/zhen/Polyorigin/speciestree/Astral/nonAdmix_gene_trees.txt -o /orange/vwhitaker/zhen/Polyorigin/speciestree/Astral/nonAdmix_Astral.tre 2>/orange/vwhitaker/zhen/Polyorigin/speciestree/Astral/out.log"""
print(script)
subprocess(script, shell=True)

In [8]:
script = """ml iq-tree
iqtree -s speciestree/concatenated.fasta -te speciestree/Astral/nonAdmix_Astral.tre -pre speciestree/Astral/nonAdmix_Astral.tre --prefix speciestree/iqtree"""
script = slurm + script
script_file = "iqtree_script.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

Submitted batch job 42217709


CompletedProcess(args=['sbatch', 'iqtree_script.sh'], returncode=0)

## Dating with LSD2

In [ ]:
ml = "\nml iq-tree"
script = "\niqtree -s speciestree/concatenated_singlecopy.fasta -m JTT+F+I+R8 --date speciestree/LSD2/date.txt --date-tip 0 -o Rchinensis --date-ci 100 -te speciestree/Astral/nonAdmix_Astral.tre --prefix speciestree/LSD2/lsd.singlecopy --redo 2>speciestree/LSD2/out.singlecopy.log"
script = slurm +ml + script
script_file = "LSD2_script_singlecopy.sh"
with open(script_file, "w") as f:
    f.write(script)
subprocess.run(['sbatch',script_file])

## Find HE using short-reads of bracteata samples

In [ ]:
## regular alignment only retain best alignment
index_dir = "/orange/vwhitaker/zhen/popgen/farr1_index/"
filtered_df = df[df.iloc[:,1].str.contains('bracteata')]
for index, row in filtered_df.iterrows():
    if os.path.isfile('./Diploid_align_octoploid/{}.bam'.format(row[0])):
        print('./Diploid_align_octoploid/{}.bam exists'.format(row[0]))
    else:    
        if pd.isna(row['File3']) == True:
            script = slurm + "\nml snap/2.0.3" + "\nsnap-aligner paired {index_dir} {file1} {file2} -pre- -so -o ./Diploid_align_octoploid/{id}.bam".format(index_dir = index_dir,file1 = row['File1'],file2=row['File2'],id=row[0])
        else:
            script = slurm + "\nml snap/2.0.3" + "\nsnap-aligner paired {index_dir} {file1} {file2} {file3} {file4} -pre- -so -o ./Diploid_align_octoploid/{id}.bam".format(index_dir = index_dir,file1 = row['File1'],file2=row['File2'],
                                                                                                                                        file3=row['File3'],file4=row['File4'],id=row[0])
        script_file = "{}_SNAP.sh".format(row[0])
        with open(script_file, "w") as f:
            f.write(script)
        result = subprocess.run(['sbatch', script_file])

In [ ]:
## get chromosome sizes using seqkit
script = """ml seqkit
seqkit fx2tab /blue/vwhitaker/fanzhen/asm/farr1.fa -n -l > farr1.chrlength.txt"""
subprocess.run(script,shell = True)

In [ ]:
#get the window file needed for coverage count
chromosome_sizes_file = "farr1.chrlength.txt"
chromosome_sizes = {}
with open(chromosome_sizes_file, 'r') as f:
    for line in f:
        chrom, size = line.strip().split('\t')
        chromosome_sizes[chrom] = int(size)

window_size = 10000  # Specify the window size here

with open('farr1.regions.bed', 'w') as bed_file:
    for chrom, size in chromosome_sizes.items():
        for start in range(0, size, window_size):
            end = min(start + window_size, size)
            bed_file.write(f'{chrom}\t{start}\t{end}\n')

In [12]:
bam_files = glob.glob("./Diploid_align_octoploid" + '/*.bam')
ml = "ml samtools\n"
script = "samtools bedcov farr1.regions.bed {} > {}"
for bam_file in bam_files: 
    script_full = slurm + ml + script.format(bam_file,bam_file+".txt")
    script_file = "{}_bedcov.sh".format(bam_file).replace("./Diploid_align_octoploid/", "")
    with open(script_file, "w") as f:
        f.write(script_full)
    result = subprocess.run(['sbatch', script_file])

Submitted batch job 37739728
Submitted batch job 37739729
Submitted batch job 37739730


In [ ]:
for index, row in filtered_df.iterrows():
    if os.path.isfile('./Diploid_align_octoploid/{}.bam'.format(row[0])):
        print('./Diploid_align_octoploid/{}.bam exists'.format(row[0]))
    else:    
        if pd.isna(row['File3']) == True:
            script = slurm + "\nml snap/2.0.3" + "\nsnap-aligner paired {index_dir} {file1} {file2} -pre- -so -o ./Diploid_align_octoploid/{id}.bam".format(index_dir = index_dir,file1 = row['File1'],file2=row['File2'],id=row[0])
        else:
            script = slurm + "\nml snap/2.0.3" + "\nsnap-aligner paired {index_dir} {file1} {file2} {file3} {file4} -pre- -so -o ./Diploid_align_octoploid/{id}.bam".format(index_dir = index_dir,file1 = row['File1'],file2=row['File2'],
                                                                                                                                        file3=row['File3'],file4=row['File4'],id=row[0])
        script_file = "{}_SNAP.sh".format(row[0])
        with open(script_file, "w") as f:
            f.write(script)
        result = subprocess.run(['sbatch', script_file])
##data analyses were done in Rscript

## Find synteny regions in F. Vesca genome for gene trees in the region

In [ ]:
##genome alignment to find Synteny regions

df1 = pd.read_csv("Diploid_align_octoploid/introgress_vesca_coords.txt",delimiter="\t",header=None)


In [ ]:
script = """
ml bcftools
bcftools view -r {chr}:{start}-{end} SNP.40w.vcf.gz -Oz > Diploid_align_octoploid/regiontree/{chr}_{start}_{end}.vcf.gz
ml python
python ../popgen/vcf2phylip/vcf2phylip/vcf2phylip.py -i Diploid_align_octoploid/regiontree/{chr}_{start}_{end}.vcf.gz 
ml iq-tree
iqtree2 -s {chr}_{start}_{end}.min4.phy -B 1000 -T AUTO --prefix Diploid_align_octoploid/regiontree/{chr}_{start}_{end} -alrt 1000 -m MFP -T AUTO -redo
"""
for index, row in df1.iterrows():
    chrid= row[0]
    start = row[1]
    end = row[2]
    script1 = slurm + script.format(chr = chrid, start = start, end = end)
    script_file = "{}_{}_tree.sh".format(chrid, start)
    with open(script_file, "w") as f:
        f.write(script1)
    result = subprocess.run(['sbatch', script_file])